Overview


In this example, you are going to evaluate a scenario of generating customer conversation summaries using generative AI to enhance service quality.

In the following cell, we are importing necessary packages and setting up a bedrock client.

In [1]:
import json
import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

Summarizing a short text with boto3

To learn detail of API request to Amazon Bedrock, this notebook introduces how to create API request and send the request via Boto3.

Request Syntax of InvokeModel in Boto3

We use InvokeModel API for sending request to a foundation model. Here is an example of API request for sending text to Amazon Titan Text Large. Inference parameters in textGenerationConfig depends on the model that you are about to use. Inference paramerters of Amazon Titan Text are:

- maxTokenCount configures the max number of tokens to use in the generated response.
- stopSequences is used to make the model stop at a desired point, such as the end of a sentence or a list. The returned response will not contain the stop sequence.
- temperature modulates the probability density function for the next tokens, implementing the temperature sampling technique. This parameter can be used to deepen or flatten the density function curve. A lower value results in a steeper curve and more deterministic responses, whereas a higher value results in a flatter curve and more random responses. (float, defaults to 0, max value is 1)
- topP controls token choices, based on the probability of the potential choices. If you set Top P below 1.0, the model considers only the most probable options and ignores less probable options. The result is more stable and repetitive completions.

response = bedrock.invoke_model(body={
                                   "inputText": "<<place your input text here>>",
                                   "textGenerationConfig": {
                                       "maxTokenCount": 4096,
                                       "stopSequences": [],
                                       "temperature":0,
                                       "topP":1
                                       },
                                },
                                modelId="amazon.titan-tg1-large", 
                                accept=accept, 
                                contentType=contentType)

Writing prompt with text to be summarized

In this notebook, you can use any short text whose tokens are less than the maximum token of a foundation model The prompt starts with an instruction "Please provide a summary of the following text."

In [2]:
prompt = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
[Agent] Hi, thank you for calling Best Internet company. This is Collin, how may I help. You?
[Customer] Hi Collin. So, I'm calling because I have been overcharged, um, my subscription to you guys is supposed to be a flat rate of 50 dollars per month but for some reason I am seeing 75 dollars on my paper bill for the month of March. So, I don't know what's going on. I would really appreciate an explanation on this.
[Agent] Sure, I'll be happy to pull up your account now and check the billing details for you. Can I have your account number together with your first and last name?
[Customer] Sure it's one two three four five and my full name is Jason Walls.
[Agent] Williams. Thank you, Jason. Just give me a few seconds to pull up your account. It was for the month of March this year, right? March 2024?
[Customer] That's right.
[Agent] Okay let me see that. Okay I have your account pulled up now and I'm only seeing a 50 charge for the month of March this year. Could you recheck your bill and review the amount?
[Customer] No. That can't be right. I am literally holding my bill right now and it says March and it says 75 dollars in big bold letters. So, may the charge be moved to Feb or April? I don't know. All I know is that you're overcharging me and it's written on my paper bill.
[Agent] Yes, Jason, I also checked your other month's charges this year from January to March 2024 and all of these months you were only charged 50 dollars. Let me just check further though your last year. Oh, although I am seeing 75 dollars charge here but this was way back last year on March 2023. Would you mind checking the year of the billing paper you're holding right now?
[Customer] Oh My Goodness! Collin, I'm so, so, sorry to have wasted your time. I really thought this was a recent bill.
[Agent] No, don't worry about it, Jason. You're fine.
[Customer] This bill was literally on my table this morning. I must have dropped it when I was throwing garbage this morning. So embarrassing, my bad.
[Agent] Oh, no problem. It happens, I mean it happens to all of us you know.
[Laughter]
[Customer] I know, right, anyway Collin I won't waste any more of your time, and thank you so much for your patience.
[Agent] You're welcome Jason. Is there anything else that I can help you with today?
[Customer] No, that's all Collin. Have a good one.
[Agent] Have a good one Jason. Thank you for calling Best Internet. Bye.
[Customer] Bye, Collin.
</text>

"""

In [3]:
body = json.dumps({"inputText": prompt, 
                   "textGenerationConfig":{
                       "maxTokenCount":4096,
                       "stopSequences":[],
                       "temperature":0,
                       "topP":1
                   },
                  }) 

Invoke foundation model via Boto3

Here sends the API request to Amazon Bedrock with specifying request parameters modelId, accept, and contentType. Following the prompt, the foundation model in Amazon Bedrock sumamrizes the text.

In [4]:
modelId = 'amazon.titan-text-express-v1' # change this to use a different model
accept = 'application/json'
contentType = 'application/json'

try:
    
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    print(response_body.get('results')[0].get('outputText'))

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

The customer calls Best Internet company to complain about being overcharged. The agent checks the customer's account and finds that the subscription is supposed to be a flat rate of 50 dollars per month, but the customer is seeing 75 dollars on their paper bill for the month of March. The agent explains that the charge is from last year and that the customer was only charged 50 dollars for the other months this year. The customer apologizes for wasting the agent's time and thanks them for their patience. The agent assures the customer that it happens to everyone and that there is nothing else that they can help them with. The customer ends the call by thanking the agent and saying goodbye.


Let's try to use Claude 3 Sonnet model for the same scenario

In [5]:
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ]
      }
    ],
    "temperature":0.5,
    "top_k":250,
    "top_p":0.5,
    "stop_sequences":[]
})

In [6]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('content')[0]['text'])

Here is a summary of the conversation:

The customer, Jason Walls, called Best Internet company thinking he was overcharged $75 instead of the usual $50 monthly fee for March 2024. The agent, Collin, looked up Jason's account and confirmed he was only charged $50 for March 2024. After some back and forth, Jason realized the $75 charge was actually from his bill from March 2023, which he had mistakenly picked up that morning. Jason apologized for the misunderstanding and wasting Collin's time. Collin was understanding and polite throughout the call. In the end, there were no issues with Jason's current billing, and the call ended amicably.


Conclusion

You have now experimented with using boto3 SDK to invoke Amazon Bedrock APIs. Using this API you have seen the use case of generating a summary of call transcript.